# **TIỀN XỬ LÝ** 

## Các bước thực hiện:
 - Kiểm tra và tiền xử lý output 
 - Tách dữ liệu thành 3 tập: train, validaton, test.
 - Tiến hành tiền xử lý


In [1]:
# IMPORT THƯ VIỆN
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

## Khám phá dữ liệu (đủ để có thể xác định câu hỏi)

In [2]:
all_houses_df = pd.read_csv('HOUSEs_info.csv', sep = '\t')
all_houses_df

,Loại hình,Phòng ngủ,Phòng tắm,Diện tích,Diện tích sử dụng,Tình hình nội thất,Loại chủ quyền,Dự án,Tháp,Giá bán,...,Q2 Thảo Điền,Căn hộ cao cấp quận 2,Chủ đầu tư Singapore Thảo Điền,Căn hộ Thảo Điền,Căn hộ hạng sang,Căn hộ view sông,Đầy đủ vệ sinh,Kệ bếp,Hoàn thiện cơ bản,Nhà trống
0,Căn hộ,3.0,2.0,88 m²,88 m²,Đầy đủ nội thất,Sổ hồng,Sky Garden,Sky Garden 2,3 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Căn hộ,3.0,2.0,90 m²,86 m²,Đầy đủ nội thất,HĐ mua bán,The Sun Avenue,Block 2,4.9 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Căn hộ,4.0,2.0,128 m²,128 m²,Nội thất cơ bản,Sổ hồng,NaN,NaN,4.25 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nhà phố,4.0,4.0,31.8 m²,76.3 m²,Không có nội thất,Sổ hồng,NaN,NaN,5.62 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nhà phố,2.0,1.0,26.5 m²,54 m²,Không có nội thất,Sổ hồng,NaN,NaN,2.4 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6930,Căn hộ,2.0,1.0,69 m²,63 m²,Nội thất cơ bản,HĐ mua bán,Vinhomes Grand Park,S3.03,2.6 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6931,Căn hộ,2.0,2.0,65.4 m²,60.7 m²,Nội thất cơ bản,HĐ mua bán,Lovera Vista Khang Điền,BLOCK C,2.05 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6932,Căn hộ,2.0,2.0,66.4 m²,66.4 m²,Đầy đủ nội thất,HĐ mua bán,Masteri Thảo Điền,T1,8 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6933,Căn hộ,2.0,2.0,67 m²,67 m²,Nội thất cơ bản,NaN,Green Star Sky Garden,NaN,2.3 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [3]:
all_houses_df.shape


(6935, 123)

### Dữ liệu có các dòng bị lặp không?

In [4]:
all_houses_df.index.duplicated().sum()

0

## Đưa ra câu hỏi cần trả lời
*Output - Giá bán -* được tính từ *input - các thông tin của hành khách -* theo công thức nào?
 Trong đồ án này, ý nghĩa thực tế của việc đi tìm câu trả lời cho câu hỏi trên giúp người dùng có được 1 sự tham khảo với 1 độ chính xác cụ thể về giá nhà dựa trên các thông tin. Qua đó, người dùng có thể dựa trên dự đoán để tự đưa ra quyết định về việc mua (bán) nhà với một mức giá hợp lý.  


## Khám phá dữ liệu 
### 1. OUTPUT TRƯỚC KHI TIỀN XỬ LÝ

In [5]:
# Cột output hiện có kiểu dữ liệu gì?
all_houses_df['Giá bán'].dtype

dtype('O')

In [6]:
# Cột output có giá trị thiếu không?
all_houses_df['Giá bán'].isna().sum()

393

#### TIỀN XỬ LÝ OUTPUT

In [7]:
# Xoá ouput có giá trị thiếu
all_houses_df = all_houses_df[all_houses_df['Giá bán'].notna()]
all_houses_df['Giá bán'].isna().sum()
all_houses_df = all_houses_df.reset_index(drop = True) 

In [8]:
# Chuẩn hoá về dạng số

# Vị trí các số hảng tỷ
mil_index = [] 
# Vị trí các số hảng triệu
bil_index = []

for i in range(0, all_houses_df.shape[0]):
    value = all_houses_df['Giá bán'][i] 
    if 'tỷ' in value:
        all_houses_df.at[i, 'Giá bán'] = value.rstrip('tỷ')
        bil_index.append(i)
    elif 'tr' in value:
        all_houses_df.at[i, 'Giá bán'] = value.rstrip(' tr')
        mil_index.append(i)

In [9]:
# Chuyển đơn vị triệu thành tỷ (đồng)
all_houses_df['Giá bán'] = all_houses_df['Giá bán'].astype(float)
for i in mil_index:
    all_houses_df.at[i, 'Giá bán'] = all_houses_df['Giá bán'][i] / 1000

### 2. SAU KHI TIỀN XỬ LÝ OUTPUT

In [10]:
# Cột output hiện có kiểu dữ liệu gì?
all_houses_df['Giá bán'].dtype

dtype('float64')

In [11]:
# Cột output có giá trị thiếu không?
all_houses_df['Giá bán'].isna().sum()

0

In [12]:
#Tỉ lệ các lớp trong cột output?
all_houses_df['Giá bán'].value_counts(normalize=True) * 100

3.50     1.605014
4.50     1.559156
3.20     1.467441
2.50     1.452155
4.00     1.421584
           ...   
4.89     0.015286
18.02    0.015286
5.56     0.015286
26.20    0.015286
7.56     0.015286
Name: Giá bán, Length: 605, dtype: float64

In [13]:
all_houses_df.describe(include="all")

,Loại hình,Phòng ngủ,Phòng tắm,Diện tích,Diện tích sử dụng,Tình hình nội thất,Loại chủ quyền,Dự án,Tháp,Giá bán,...,Q2 Thảo Điền,Căn hộ cao cấp quận 2,Chủ đầu tư Singapore Thảo Điền,Căn hộ Thảo Điền,Căn hộ hạng sang,Căn hộ view sông,Đầy đủ vệ sinh,Kệ bếp,Hoàn thiện cơ bản,Nhà trống
count,6542,6415.000000,6387.000000,6488,5331,6228,4641,5321,3073,6542.000000,...,1,1,1,1,1,1,1,1,1,1
unique,15,NaN,NaN,1375,1257,3,4,381,236,NaN,...,1,1,1,1,1,1,1,1,1,1
top,Căn hộ,NaN,NaN,59 m²,63 m²,Nội thất cơ bản,HĐ mua bán,Vinhomes Grand Park,A,NaN,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
freq,5289,NaN,NaN,200,164,2406,3344,862,103,NaN,...,1,1,1,1,1,1,1,1,1,1
mean,NaN,2.157599,1.889620,NaN,NaN,NaN,NaN,NaN,NaN,4.232327,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,1.111613,1.033057,NaN,NaN,NaN,NaN,NaN,NaN,4.471415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.690000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,2.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,2.320000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,2.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,3.160000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,3.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,4.600000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


   ## Tiền xử lí( tách các tập)

In [14]:
# Tách X và y
y_sr = all_houses_df["Giá bán"] # sr là viết tắt của series
X_df = all_houses_df.drop("Giá bán", axis=1)